# S1 - Federated Learning for IoT and Smart Cities

## Code Structure

1. Setup enviroment by importing libs and dataset
2. Preprocess the data by
- Flatten the data
- Selects clients (if needed)
- Batch and create dict
3. Prepare the model
- Keras model 
- Create layers
- Cast keras model to tff-model
4. Build and test FL algorithm
- Initialization of model on server.
Then as a "loop" :
- server-to-client broadcast step.
- local client update step.
- client-to-server upload step.
- server update step

### 1. Setup enviroment

In [1]:
import nest_asyncio
import collections
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff # this might register as non-resolved but seems to work
np.random.seed(0)
nest_asyncio.apply()
# check if tff works correctly
tff.federated_computation(lambda: 'Hello, World!')()

2021-11-29 16:33:29.979614: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-29 16:33:29.979671: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-11-29 16:33:36.762498: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-29 16:33:36.763082: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-29 16:33:36.763233: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-94AKSBPP): /proc/driver/nvidia/version does not exist
2021-11-29 16:33:36.764256: I tensorflow/core/platform/cpu_fe

b'Hello, World!'

In [2]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

### 2. Process the data

Flatten batches of data 

In [3]:
NUM_CLIENTS = 10
BATCH_SIZE = 20

def preprocess(dataset):

  def batch_format_fn(element):
    """Flatten a batch of EMNIST data and return a (features, label) tuple."""
    return (tf.reshape(element['pixels'], [-1, 784]), 
            tf.reshape(element['label'], [-1, 1]))

  return dataset.batch(BATCH_SIZE).map(batch_format_fn)

In [ ]:
client_ids = sorted(emnist_train.client_ids)[:NUM_CLIENTS]
federated_train_data = [preprocess(emnist_train.create_tf_dataset_for_client(x))
  for x in client_ids
]